In [48]:
import pandas as pd
import numpy as np
import re

In [49]:
livros_pre_2017 = pd.read_csv("livros_pre_2017.csv", index_col = "Unnamed: 0", parse_dates=["Data de edição"])
livros_pos_2017 = pd.read_csv("livros pos 2017.csv", index_col = "Unnamed: 0", parse_dates=["Data de edição:"])

In [50]:
#lista isbn
isbn_list_final = list(livros_pos_2017['ISBN:']) + list(livros_pre_2017['ISBN'])
type(isbn_list_final)

with open('isbn-list.txt', 'w') as filehandle:
    for listitem in isbn_list_final:
        filehandle.write('%s\n' % listitem)

## Datas de Edição

In [51]:
def clean_dates(date):
    if type(date) is not str:
        return None
    date = re.search(r"\d{4}", date)
    return date[0] if date is not None else None

livros_pos_2017["Data de edição:"] = livros_pos_2017["Data de edição:"].apply(clean_dates)
livros_pre_2017["Data de edição"] = livros_pre_2017["Data de edição"].apply(clean_dates)

In [52]:
datas_pos_2017 = livros_pos_2017["Data de edição:"].value_counts()
datas_pre_2017 = livros_pre_2017["Data de edição"].value_counts()
datas_pre_2017 = pd.DataFrame(datas_pre_2017)
datas_pre_2017 = datas_pre_2017.reset_index()
datas_pos_2017 = pd.DataFrame(datas_pos_2017)
datas_pos_2017 = datas_pos_2017.reset_index()
datas_pos_2017.columns = datas_pre_2017.columns

In [53]:
todas_datas = datas_pre_2017.merge(datas_pos_2017, how="outer", on="index")
todas_datas = todas_datas.fillna(0)
todas_datas["Livros editados"] = todas_datas["Data de edição_x"] + todas_datas["Data de edição_y"]
todas_datas = todas_datas.drop(["Data de edição_x", "Data de edição_y"], axis=1)
todas_datas = todas_datas.astype("int")
#todas_datas.to_csv("datas-edicao-final.csv")

In [54]:
todas_datas["Livros editados"].sum()

8301

In [55]:
#livros editados depois do início do PNL

datas_pos_2006 = todas_datas.loc[(todas_datas["index"] > 2005) & (todas_datas["index"] < 2021)]
datas_pos_2006.describe()

,index,Livros editados
count,15.000000,15.000000
mean,2013.000000,482.533333
std,4.472136,173.275943
min,2006.000000,173.000000
25%,2009.500000,384.500000
50%,2013.000000,471.000000
75%,2016.500000,544.000000
max,2020.000000,888.000000


In [56]:
(todas_datas["Livros editados"].sum() - datas_pos_2006["Livros editados"].sum()) / todas_datas["Livros editados"].sum()

0.12805686061920252

## Obras Traduzidas

In [57]:
livros_pre_2017 = livros_pre_2017.drop_duplicates(subset=['Título', 'Autor'], keep='first')

In [58]:
livros_pre_2017["Línguas01"].value_counts()

Português    4966
Inglês        778
Outra           1
Name: Línguas01, dtype: int64

In [59]:
livros_pos_2017["Língua:"].value_counts()

Português              2312
N/R                      26
Inglês                    3
Outra                     2
Português; Espanhol       1
português                 1
Espanhol                  1
Name: Língua:, dtype: int64

In [60]:
livros_pos_2017.isna().sum()

ISBN:                  0
Título:                2
Autor(es):             0
Ilustrador(es):      962
Tradutor(es)         918
Editora:               0
Data de edição:        5
Palavras-chave:        4
Idade:                 1
Nível de Leitura:      0
Tema:                  0
Formato:               0
Língua:                3
Ano PNL                0
dtype: int64

In [61]:
livros_pre_2017["Tradutor"].isna().sum()

3703

In [62]:
tradutores_pos_2017 = livros_pos_2017["Tradutor(es)"].value_counts()
#tradutores_pos_2017.to_csv("tradutores_pos_2017.csv")

In [63]:
tradutores_pos_2017_limpo = pd.read_csv("tradutores_pos_2017.csv")
tradutores_pos_2017_limpo["Tradutor(es)"].sum()

751

In [64]:
tradutores_pre_2017 = livros_pre_2017["Tradutor"].value_counts()
tradutores_pre_2017.to_csv("tradutores_pre_2017.csv")
tradutores_pre_2017.sum()/len(tradutores_pre_2017)

2.0039254170755645

In [65]:
livros_pre_2017.isna().sum()

ISBN                    49
Título                   0
Autor                  105
Ilustrador            2541
Tradutor              3703
Data de edição         112
Editora                  7
Idade02                  1
Idade02.1             5023
Nível de leitura01       0
Nível de leitura02    4913
Temas01                 54
Temas02               5118
Temas03               5733
Formato01             3747
Formato02             5712
Línguas01                0
Línguas02             5702
dtype: int64

In [66]:
livros_pos_2017.isna().sum()

ISBN:                  0
Título:                2
Autor(es):             0
Ilustrador(es):      962
Tradutor(es)         918
Editora:               0
Data de edição:        5
Palavras-chave:        4
Idade:                 1
Nível de Leitura:      0
Tema:                  0
Formato:               0
Língua:                3
Ano PNL                0
dtype: int64

In [67]:
#correcção manual de temas

livros_pre_2017.loc[livros_pre_2017['Temas01'] == 'Ciências & tecnologia', "Temas01"] = 'Ciência e Tecnologia'
livros_pre_2017.loc[livros_pre_2017['Temas01'] == 'Ciências & Tecnologia', "Temas01"] = 'Ciência e Tecnologia'
livros_pre_2017.loc[livros_pre_2017['Temas01'] == 'Ciências e Tecnologia', "Temas01"] = 'Ciência e Tecnologia'
livros_pre_2017.loc[livros_pre_2017['Temas01'] == 'Ciências e Tecnologias', "Temas01"] = 'Ciência e Tecnologia'
livros_pre_2017.loc[livros_pre_2017['Temas01'] == 'Poesa', "Temas01"] = 'Poesia'
livros_pre_2017.loc[livros_pre_2017['Temas01'] == 'Vida prática', "Temas01"] = 'Vida Prática'
livros_pre_2017.loc[livros_pre_2017['Temas01'] == 'Biografias', "Temas01"] = 'Biografia'

In [68]:
livros_pre_2017.loc[livros_pre_2017['Temas02'] == 'Ciências & tecnologia', "Temas02"] = 'Ciência e Tecnologia'
livros_pre_2017.loc[livros_pre_2017['Temas02'] == 'Ciências & Tecnologia', "Temas02"] = 'Ciência e Tecnologia'
livros_pre_2017.loc[livros_pre_2017['Temas02'] == 'Ciências e Tecnologia', "Temas02"] = 'Ciência e Tecnologia'
livros_pre_2017.loc[livros_pre_2017['Temas02'] == 'Ciâncias & Tecnologia', "Temas02"] = 'Ciência e Tecnologia'
livros_pre_2017.loc[livros_pre_2017['Temas02'] == 'Ciências & Tecnlogia', "Temas02"] = 'Ciência e Tecnologia'
livros_pre_2017.loc[livros_pre_2017['Temas02'] == 'Cultura e sociedade', "Temas02"] = 'Cultura e Sociedade'

livros_pre_2017.loc[livros_pre_2017['Temas02'] == 'Poesa', "Temas02"] = 'Poesia'
livros_pre_2017.loc[livros_pre_2017['Temas02'] == 'Vida prática', "Temas02"] = 'Vida Prática'
livros_pre_2017.loc[livros_pre_2017['Temas02'] == 'Biografias', "Temas02"] = 'Biografia'


In [69]:
#livros_pre_2017.groupby(["Temas01","Temas02"]).agg({"Temas01":"count", "Temas01":"count"}).to_csv("temas_pre_2017.csv")

In [70]:
livros_pre_2017['Temas01'].value_counts()

Literatura              3742
Cultura e Sociedade      765
Ciência e Tecnologia     655
Poesia                   263
Artes                    191
Vida Prática              38
Biografia                 20
Banda Desenhada            8
Ensaio                     8
Livro jogo                 1
Name: Temas01, dtype: int64

In [71]:
livros_pre_2017['Temas02'].value_counts()

Literatura              328
Cultura e Sociedade     134
Poesia                   49
Biografia                47
Ciência e Tecnologia     31
Artes                    20
Ensaio                   10
Vida Prática              6
Livro jogo                2
Name: Temas02, dtype: int64

In [72]:
livros_pos_2017["Nível de Leitura:"].unique()

array(['Leitura autónoma', 'Leitura mediana;Leitura autónoma', 'N/R',
       'Leitura mediana', 'Leitura inicial', 'Pré-leitura',
       'Leitura inicial;Leitura mediana;Leitura autónoma',
       'Leitura inicial;Leitura mediana', 'Pré-leitura;Leitura inicial',
       'Leitura inicial;Leitura autónoma',
       'Pré-leitura;Leitura inicial;Leitura autónoma',
       'Pré-leitura;Leitura autónoma', 'Leitura fluente',
       'Leitura mediana;Leitura fluente',
       'Pré-leitura;Leitura inicial;Leitura mediana',
       'Leitura inicial;Leitura mediana;Leitura fluente',
       'Pré-leitura;Leitura inicial;Leitura fluente',
       'Leitura inicial, Leitura mediana',
       'Leitura fluente;Leitura mediana;', 'Leitura fluente;',
       'Pré-leitura;Leitura fluente;', 'Pré-leitura;Leitura inicial;',
       'Pré-leitura;', 'Leitura inicial;Leitura mediana;',
       'Leitura inicial;', 'Pré-leitura;Leitura fluente;Leitura inicial;',
       'Leitura fluente;Leitura inicial;Leitura mediana;',
    

In [104]:
temas_pos_2017 = livros_pos_2017['Tema:'].str.split(';', expand=True).rename(columns=lambda x: f"temas{x+1}")
idade_pos_2017 = livros_pos_2017['Idade:'].str.split(';', expand=True).rename(columns=lambda x: f"idade{x+1}")
niveis_pos_2017 = livros_pos_2017["Nível de Leitura:"].str.split(';', expand=True).rename(columns=lambda x: f"nivel_leitura{x+1}")


In [105]:
livros_pos_2017_limpo = pd.concat([livros_pos_2017, temas_pos_2017.reindex(livros_pos_2017.index)], axis=1)
livros_pos_2017_limpo = pd.concat([livros_pos_2017_limpo, idade_pos_2017.reindex(livros_pos_2017_limpo.index)], axis=1)
livros_pos_2017_limpo = pd.concat([livros_pos_2017_limpo, niveis_pos_2017.reindex(livros_pos_2017_limpo.index)], axis=1)
livros_pos_2017_limpo

,ISBN:,Título:,Autor(es):,Ilustrador(es):,Tradutor(es),Editora:,Data de edição:,Palavras-chave:,Idade:,Nível de Leitura:,...,temas4,idade1,idade2,idade3,idade4,idade5,nivel_leitura1,nivel_leitura2,nivel_leitura3,nivel_leitura4
1,978-989-644-448-8,"Outras Mentes - O Polvo, o Mar e a Origem da C...","Godfrey-Smith, P.",n. d.,"Garcia Rosado, P.",N/R,2017,Mente; Consciência; Evolução,15-18 anos;Maiores de 18 anos,Leitura autónoma,...,None,15-18 anos,Maiores de 18 anos,None,None,None,Leitura autónoma,None,None,None
2,9789896168117,Poções e paixões,"André, João Paulo",não há,não há,N/R,2018,"música, química, ópera",15-18 anos;Maiores de 18 anos,Leitura autónoma,...,None,15-18 anos,Maiores de 18 anos,None,None,None,Leitura autónoma,None,None,None
3,978-989-644-434-1,"Nós, os Micróbios e uma Visão ALargada da Vida","Young, E.",n. d.,"Santos, J.L.",N/R,2017,Micróbios; Microbioma; Simbiose,15-18 anos;Maiores de 18 anos,Leitura autónoma,...,None,15-18 anos,Maiores de 18 anos,None,None,None,Leitura autónoma,None,None,None
4,9789896417529,Preparação para a Noite,"Rocha, Jaime",N/R,N/R,N/R,2017,Poesia; estilo; individualidade; coerência.,N/R,Leitura autónoma,...,None,N/R,None,None,None,None,Leitura autónoma,None,None,None
5,9789722361569,"Os Contos de Beedle, o Bardo","Rowling, J.K.",S/ ilustrar.,s/ indic.,N/R,2018,feitiçaria; Harry Potter,9-11 anos,Leitura autónoma,...,None,9-11 anos,None,None,None,None,Leitura autónoma,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,978-972-25-3934-0,A Menina com os Olhos Ocupados,André Carrilho,André Carrilho,n.a,Bertrand Editora,2020,"telemóveis, comunicação, família",9-11 anos;12-14 anos;,Leitura fluente;,...,None,9-11 anos,12-14 anos,,None,None,Leitura fluente,,None,None
253,978-989-8688-58-3,A gata e a fábula,"Botelho, Fernanda",Sem ilustração,Sem radudor,Abysmo,2018,Insubmissão; conformismo; liberdade,Maiores de 18 anos;15-18 anos;,Leitura fluente;,...,None,Maiores de 18 anos,15-18 anos,,None,None,Leitura fluente,,None,None
254,978-989-783-017-4,NaN,"Handke, Peter",Sem ilustrador,"Lopes, Alexandra",Relógio D'Água,2020,felicidade; Beleza; Reflexão,Maiores de 18 anos;15-18 anos;,Leitura fluente;,...,None,Maiores de 18 anos,15-18 anos,,None,None,Leitura fluente,,None,None
255,978-989-722-709-7,O Que É Amar um País,José Tolentino Mendonça,n.a,n.a,Quetzal Editores,2020,"esperança, reflexão, vida",Maiores de 18 anos;15-18 anos;,Leitura fluente;,...,None,Maiores de 18 anos,15-18 anos,,None,None,Leitura fluente,,None,None


In [107]:
livros_pre_2017.columns

Index(['ISBN', 'Título', 'Autor', 'Ilustrador', 'Tradutor', 'Data de edição',
       'Editora', 'Idade02', 'Idade02.1', 'Nível de leitura01',
       'Nível de leitura02', 'Temas01', 'Temas02', 'Temas03', 'Formato01',
       'Formato02', 'Línguas01', 'Línguas02'],
      dtype='object')

In [109]:
livros_pos_2017_limpo.columns

Index(['ISBN:', 'Título:', 'Autor(es):', 'Ilustrador(es):', 'Tradutor(es)',
       'Editora:', 'Data de edição:', 'Palavras-chave:', 'Idade:',
       'Nível de Leitura:', 'Tema:', 'Formato:', 'Língua:', 'Ano PNL',
       'temas1', 'temas2', 'temas3', 'temas4', 'idade1', 'idade2', 'idade3',
       'idade4', 'idade5', 'nivel_leitura1', 'nivel_leitura2',
       'nivel_leitura3', 'nivel_leitura4'],
      dtype='object')

In [163]:
#temas e idades principais

idades_temas_pos_2017 = livros_pos_2017_limpo.groupby(["idade1","temas1"])["Título:"].count()
idades_temas_pre_2017 = livros_pre_2017.groupby(["Idade02","Temas01"])["Título"].count()

idades_temas_pre_2017.rename(index={'maiores de 18 anos': "Maiores de 18 anos", "Artes":"Arte", "Ciências e Tecnologias":"Ciência e Tecnologia"}, inplace=True)

idades_temas_limpo = pd.concat([idades_temas_pre_2017, idades_temas_pos_2017], axis=1)


idades_temas_limpo.fillna(0, inplace=True)
idades_temas_limpo["livros"] = idades_temas_limpo["Título:"] + idades_temas_limpo["Título"]
idades_temas_limpo["livros"].astype("int")
idades_temas_limpo = idades_temas_limpo.drop(columns=["Título:", "Título"])
idades_temas_limpo.to_csv("temas_idades_principais.csv")


In [168]:
niveis_pos_2017.count()

nivel_leitura1    2349
nivel_leitura2    1226
nivel_leitura3     248
nivel_leitura4      27
dtype: int64

In [173]:
livros_pos_2017_limpo.groupby(["nivel_leitura1","nivel_leitura2"]).agg({"Título:":"count"})

Título:
nivel_leitura1  nivel_leitura2           
Leitura fluente                       331
                Leitura inicial         2
                Leitura mediana        59
Leitura inicial                        31
                Leitura autónoma        1
                Leitura mediana       178
                Pré-leitura            15
Leitura mediana                        21
                Leitura autónoma       91
                Leitura fluente       323
                Leitura inicial        26
Pré-leitura                            11
                Leitura autónoma        2
                Leitura fluente         2
                Leitura inicial       131

In [197]:
niveis_leitura = pd.DataFrame(livros_pos_2017_limpo["nivel_leitura1"].value_counts())
niveis_leitura["pre2017"] = pd.Series(livros_pre_2017["Nível de leitura01"].value_counts())
niveis_leitura.fillna(0,inplace=True)
niveis_leitura.astype("int")
niveis_leitura["total"] = niveis_leitura["nivel_leitura1"] + niveis_leitura["pre2017"]
niveis_leitura.to_csv("niveis_leitura.csv")

In [76]:
'''relacao_temas = livros_pos_2017_limpo.groupby(["Temas1","Temas2"]).agg({"Temas1":"count"})
relacao_temas_pre_2017 = livros_pre_2017.groupby(["Temas01","Temas02"]).agg({"Temas01":"count"})#.to_csv("temas_pre_2017.csv")

relacao_temas.rename(columns={"Temas1":"contagem"})
relacao_temas_pre_2017.rename(columns={"Temas01":"contagem"})

relacao_temas'''

Temas1
Temas1       Temas2                      
Arte                                   20
             Banda Desenhada            4
             Biografia                  9
             Cultura e Sociedade       23
             Literatura                10
...                                   ...
Vida Prática                           19
             Banda Desenhada            4
             Ciência e Tecnologia       4
             Cultura e Sociedade        3
             Literatura                 8

[73 rows x 1 columns]

In [78]:
autores_pre_2017 = livros_pre_2017["Autor"].value_counts()
autores_pre_2017.to_csv("autores_pre_2017.csv")

In [79]:
autores_pos_2017 = livros_pos_2017["Autor(es):"].value_counts()
autores_pos_2017.to_csv("autores_pos_2017.csv")

In [80]:
livros_pre_2017.columns

Index(['ISBN', 'Título', 'Autor', 'Ilustrador', 'Tradutor', 'Data de edição',
       'Editora', 'Idade02', 'Idade02.1', 'Nível de leitura01',
       'Nível de leitura02', 'Temas01', 'Temas02', 'Temas03', 'Formato01',
       'Formato02', 'Línguas01', 'Línguas02'],
      dtype='object')

In [81]:
livros_pos_2017.columns

Index(['ISBN:', 'Título:', 'Autor(es):', 'Ilustrador(es):', 'Tradutor(es)',
       'Editora:', 'Data de edição:', 'Palavras-chave:', 'Idade:',
       'Nível de Leitura:', 'Tema:', 'Formato:', 'Língua:', 'Ano PNL'],
      dtype='object')

In [82]:
'''livros = [livros_pos_2017,livros_pre_2017]
todos_livros = pd.concat(livros)
todos_livros'''

'livros = [livros_pos_2017,livros_pre_2017]\ntodos_livros = pd.concat(livros)\ntodos_livros'